<a href="https://colab.research.google.com/github/Troyanovsky/awesome-TTS-Colab/blob/main/GLM_TTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🗣️ GLM-TTS Colab

## 📄 Description

This notebook runs **GLM-TTS**, a **LLM-powered text-to-speech system** featuring **zero-shot voice cloning**, **RL-enhanced emotion control**, and **streaming real-time synthesis**.  
Built with a **two-stage architecture**—speech token generation via an LLM and waveform synthesis via Flow Matching—GLM-TTS produces **high-fidelity, expressive**, and **bilingual (EN/ZH)** speech suitable for interactive applications.

**Capabilities:**  
Zero-Shot Voice Cloning (3–10s), RL-Tuned Emotion & Prosody, High-Quality Speech Generation, Phoneme-Level Control, Streaming / Low-Latency TTS, Chinese–English Mixed Input

---

## How to use

* Adjust the text/audio inputs in the provided fields  
* Expand and run the section that you need. Note that you may need to restart the session after the `pip install` cells and then run following cells for libraries to work properly.

---

## ⚙️ Model Highlights

* 🗣 **Zero-shot cloning** – reproduce a speaker from a few seconds of audio  
* 🎭 **Emotion-optimized prosody** – Multi-reward RL (GRPO) improves expressiveness  
* 🔤 **Hybrid phoneme+text input** – precise control over pronunciation  
* ⚡ **Streaming inference** – supports real-time generation for interactive settings  
* 🌏 **Bilingual capability** – robust for Chinese, English, and mixed-language text  

---

## 🧠 Model Details

* **Architecture:** LLM-based speech token generator + Flow Matching vocoder  
* **Supported Languages:** English, Chinese, mixed text  
* **Voice Cloning:** 3–10 seconds prompt audio  
* **Control Features:** Emotion, prosody, phoneme-level instructions  
* **Performance:** Low-latency streaming suitable for live applications  
* **Format:** Available via Hugging Face  

---

## 🔗 Resources

* **GitHub Repository:** https://github.com/zai-org/GLM-TTS  
* **Model Availability:** https://huggingface.co/zai-org/GLM-TTS  

---

## 🎙️ Explore More TTS Models

Looking for more cutting-edge voice models?  
👉 Check out the full collection: [awesome-TTS-Colab](https://github.com/Troyanovsky/awesome-TTS-Colab)

## TTS (using default example audio)

In [ ]:
# Setup: Clone repo, install deps, download checkpoints

!git clone https://github.com/zai-org/GLM-TTS.git
%cd /content/GLM-TTS

# Remove any line containing WeTextProcessing from requirements.txt
import re

req_path = "requirements.txt"

with open(req_path, "r") as f:
    lines = f.readlines()

# Filter out the problematic line(s)
cleaned = []
for line in lines:
    if not re.search(r'wetextprocessing', line, re.IGNORECASE):
        cleaned.append(line)

with open(req_path, "w") as f:
    f.writelines(cleaned)

In [ ]:
# ⚠️ You may need to restart session after installing these libraries, then run following cells
!pip install -r requirements.txt
!pip install -U "huggingface-hub>=0.34.0,<1.0"
!pip install WeTextProcessing

In [ ]:
from huggingface_hub import snapshot_download

# Download full GLM-TTS checkpoint into ./ckpt
snapshot_download(
    "zai-org/GLM-TTS",
    local_dir="ckpt",
    local_dir_use_symlinks=False,
)

In [1]:
# Text that GLM-TTS will synthesize
input_text = "Hello, this is Jiayan from GLM-TTS. How are you doing?"  #@param {type:"string"}

# Reference text that matches the built-in reference audio speaker
prompt_text = "I wonder if you'd like to have a burger with me."  #@param {type:"string"}

# Built-in reference audio shipped with the repo
prompt_wav_path = "examples/prompt/jiayan_en.wav"  #@param {type:"string"}

# Inference options
sample_rate = 24000  #@param {type:"integer"}
use_phoneme = False  #@param {type:"boolean"}
use_cache = True     #@param {type:"boolean"}
seed = 0             #@param {type:"integer"}


In [ ]:
%cd /content/GLM-TTS

import torch
from IPython.display import Audio, display

# Import GLM-TTS helpers from the repo
from glmtts_inference import load_models, generate_long, DEVICE

# Load all frontends + models (LLM + Flow)
frontend, text_frontend, speech_tokenizer, llm, flow = load_models(
    use_phoneme=use_phoneme,
    sample_rate=sample_rate,
)

def glmtts_synthesize(
    prompt_wav: str,
    prompt_text: str,
    synth_text: str,
    sample_rate: int = 24000,
    use_phoneme: bool = False,
    seed: int = 0,
    use_cache: bool = True,
):
    """Minimal wrapper around generate_long for a single utterance."""
    # Normalize text
    prompt_text_norm = text_frontend.text_normalize(prompt_text)
    synth_text_norm = text_frontend.text_normalize(synth_text)

    # Extract tokens & features from prompt
    prompt_text_token = frontend._extract_text_token(prompt_text_norm + " ")
    prompt_speech_token = frontend._extract_speech_token([prompt_wav])
    speech_feat = frontend._extract_speech_feat(prompt_wav, sample_rate=sample_rate)
    embedding = frontend._extract_spk_embedding(prompt_wav)

    cache_speech_token = [prompt_speech_token.squeeze().tolist()]
    flow_prompt_token = torch.tensor(cache_speech_token, dtype=torch.int32).to(DEVICE)

    # LLM cache (for longer text, can reuse history)
    cache = {
        "cache_text": [prompt_text_norm],
        "cache_text_token": [prompt_text_token],
        "cache_speech_token": cache_speech_token,
        "use_cache": use_cache,
    }

    uttid = "simple_demo"

    # Core generation
    tts_speech, _, _, _ = generate_long(
        frontend=frontend,
        text_frontend=text_frontend,
        llm=llm,
        flow=flow,
        text_info=[uttid, synth_text_norm],
        cache=cache,
        embedding=embedding,
        seed=seed,
        flow_prompt_token=flow_prompt_token,
        speech_feat=speech_feat,
        device=DEVICE,
        use_phoneme=use_phoneme,
    )

    return tts_speech


# Run synthesis
tts_speech = glmtts_synthesize(
    prompt_wav=prompt_wav_path,
    prompt_text=prompt_text,
    synth_text=input_text,
    sample_rate=sample_rate,
    use_phoneme=use_phoneme,
    seed=seed,
    use_cache=use_cache,
)

In [ ]:
# Play audio directly from tensor
waveform = tts_speech.squeeze().cpu().numpy()
display(Audio(waveform, rate=sample_rate))

## Voice Clone

In [ ]:
# Setup: Clone repo, install deps, download checkpoints

!git clone https://github.com/zai-org/GLM-TTS.git
%cd /content/GLM-TTS

# Remove any line containing WeTextProcessing from requirements.txt
import re

req_path = "requirements.txt"

with open(req_path, "r") as f:
    lines = f.readlines()

# Filter out the problematic line(s)
cleaned = []
for line in lines:
    if not re.search(r'wetextprocessing', line, re.IGNORECASE):
        cleaned.append(line)

with open(req_path, "w") as f:
    f.writelines(cleaned)

In [ ]:
# ⚠️ You may need to restart session after installing these libraries, then run following cells
!pip install -r requirements.txt
!pip install -U "huggingface-hub>=0.34.0,<1.0"
!pip install WeTextProcessing

In [ ]:
from huggingface_hub import snapshot_download

# Download full GLM-TTS checkpoint into ./ckpt
snapshot_download(
    "zai-org/GLM-TTS",
    local_dir="ckpt",
    local_dir_use_symlinks=False,
)

In [ ]:
# Reference text that matches the built-in reference audio speaker
prompt_text = "In short, we embarked on, a mission to make America great again, for all Americans."  #@param {type:"string"}

# Upload reference audio for cloning
prompt_wav_path = "/content/GLM-TTS/examples/prompt/uploaded.wav"

from google.colab import files
import os
import shutil

# Ensure the directory exists
os.makedirs(os.path.dirname(prompt_wav_path), exist_ok=True)

# Prompt user upload
uploaded = files.upload()

# Validate upload
if not uploaded:
    raise RuntimeError("No file was uploaded.")

# Take the first uploaded file
uploaded_filename = next(iter(uploaded))

# Basic validation: ensure it's a WAV file
if not uploaded_filename.lower().endswith(".wav"):
    raise ValueError("Uploaded file is not a .wav file.")

# Save to desired path
with open(prompt_wav_path, "wb") as f:
    f.write(uploaded[uploaded_filename])

print(f"File saved to: {prompt_wav_path}")


In [2]:
# Text that GLM-TTS will synthesize
input_text = "How is this voice-cloning quality? Does it sound good?"  #@param {type:"string"}

# Inference options
sample_rate = 24000  #@param {type:"integer"}
use_phoneme = False  #@param {type:"boolean"}
use_cache = True     #@param {type:"boolean"}
seed = 0             #@param {type:"integer"}

In [ ]:
%cd /content/GLM-TTS

import torch
from IPython.display import Audio, display

# Import GLM-TTS helpers from the repo
from glmtts_inference import load_models, generate_long, DEVICE

# Load all frontends + models (LLM + Flow)
frontend, text_frontend, speech_tokenizer, llm, flow = load_models(
    use_phoneme=use_phoneme,
    sample_rate=sample_rate,
)

def glmtts_synthesize(
    prompt_wav: str,
    prompt_text: str,
    synth_text: str,
    sample_rate: int = 24000,
    use_phoneme: bool = False,
    seed: int = 0,
    use_cache: bool = True,
):
    """Minimal wrapper around generate_long for a single utterance."""
    # Normalize text
    prompt_text_norm = text_frontend.text_normalize(prompt_text)
    synth_text_norm = text_frontend.text_normalize(synth_text)

    # Extract tokens & features from prompt
    prompt_text_token = frontend._extract_text_token(prompt_text_norm + " ")
    prompt_speech_token = frontend._extract_speech_token([prompt_wav])
    speech_feat = frontend._extract_speech_feat(prompt_wav, sample_rate=sample_rate)
    embedding = frontend._extract_spk_embedding(prompt_wav)

    cache_speech_token = [prompt_speech_token.squeeze().tolist()]
    flow_prompt_token = torch.tensor(cache_speech_token, dtype=torch.int32).to(DEVICE)

    # LLM cache (for longer text, can reuse history)
    cache = {
        "cache_text": [prompt_text_norm],
        "cache_text_token": [prompt_text_token],
        "cache_speech_token": cache_speech_token,
        "use_cache": use_cache,
    }

    uttid = "simple_demo"

    # Core generation
    tts_speech, _, _, _ = generate_long(
        frontend=frontend,
        text_frontend=text_frontend,
        llm=llm,
        flow=flow,
        text_info=[uttid, synth_text_norm],
        cache=cache,
        embedding=embedding,
        seed=seed,
        flow_prompt_token=flow_prompt_token,
        speech_feat=speech_feat,
        device=DEVICE,
        use_phoneme=use_phoneme,
    )

    return tts_speech

In [5]:
# Run synthesis
tts_speech = glmtts_synthesize(
    prompt_wav=prompt_wav_path,
    prompt_text=prompt_text,
    synth_text=input_text,
    sample_rate=sample_rate,
    use_phoneme=use_phoneme,
    seed=seed,
    use_cache=use_cache,
)

In [ ]:
# Play audio directly from tensor
waveform = tts_speech.squeeze().cpu().numpy()
display(Audio(waveform, rate=sample_rate))